# 3.6 손글씨 숫자 인식

학습 과정은 생략하고, 추론 과정만 구현하기  
추론과정: 순전파(forward propagation)  

## 3.6.1 MNIST 데이터셋

In [3]:
import sys, os
sys.path.append(os.pardir) # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from dataset.mnist import load_mnist

(X_train, t_train), (X_test, t_test) = load_mnist(flatten = True, normalize = False)

In [4]:
# 각 데이터의 형상 출력

print(X_train.shape)
print(t_train.shape)
print(X_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [5]:
from PIL import Image
import numpy as np

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

img = X_train[0]
label = t_train[0]
print(label)
print(img.shape)

print(img.shape)
img = img.reshape(28, 28)
print(img.shape)

img_show(img)

5
(784,)
(784,)
(28, 28)


## 3.6.2 신경망의 추론 정리

이미지의 크기가 28*28 = 784이므로 입력층은 784개  
출력층은 0~9까지를 구별하는 문제이기 때문에 10개  
 3.6.2 신경망의 추론 정리

이미지의 크기가 28*28 = 784이므로 입력층은 784개  
출력층은 0~9까지를 구별하는 문제이기 때문에 10개  
은닉층은 첫번째 은닉층에 50개, 두번째 은닉층에는 100개의 뉴런을 배치할 것이다. (여기서 50개, 100개는 임의로 정한 값이다. )

In [6]:
import pickle

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test


def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network


def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y

In [7]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x)) 
    
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a-c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y   
    
x, t = get_data()
network = init_network()
accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p= np.argmax(y) # 확률이 가장 높은 원소의 인덱스를 얻는다.
    if p == t[i]:
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


이미지를 한장씩꺼내 predict() 함수를 이용해 확률을 해석  
목표: 99%  
load_mnist를 할 때 normalize로 전처리 작업을 함

지를 한장씩꺼내 predict() 함수를 이용해 확률을 해석  
목표: 99%  
load_mnist를 할 때 normalize로 전처리 작업을 함

## 3.6.3 배치 처리

In [8]:
x, _ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']

print(x.shape)
print(x[0].shape)
print(W1.shape)
print(W2.shape)
print(W3.shape)

(10000, 784)
(784,)
(784, 50)
(50, 100)
(100, 10)


하나로 묶은 입력 데이터를 배치(batch)라 한다.  
배치 처리는 더 효율적으로 구현이 가능해서 학습 속도를 증가시킬 수 있다.  
배치 처리를 구현해 보자.

In [9]:
x, t = get_data()
network = init_network()

batch_size = 100 # 배치 처리
accuracy_cnt = 0

for i in range(0,len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis = 1)
    accuracy_cnt += np.sum(p==t[i:i+batch_size])

print("Accuracy" + str(float(accuracy_cnt) / len(x)))

Accuracy0.9352
